# Capstone Term 2 - Project

* Marcos Bittencourt
---
* Contributors
    * Todd McCullough [Git](https://github.com/tamccullough)
    * Savya Sharma [Git](https://github.com/SavyaSharma)
    * Marko Topitch [Git](https://github.com/TopMarko)
---

### Load the Model and Data

##### Import the Needed Modules

In [1]:
import pandas as pd
import numpy as np
import heapq
import pickle

##### Load the production ready model

In [4]:
filename = 'recip-eeze-flask/recipeeze/model/recipes_recommender_model.sav'
rr_model = pickle.load(open(filename, 'rb'))

##### Import Data

In [5]:
recipes_df = pd.read_csv('recip-eeze-flask/recipeeze/datasets/rr-recipes.csv')
users_df = pd.read_csv('recip-eeze-flask/recipeeze/datasets/rr-users.csv')
ratings_df = pd.read_csv('recip-eeze-flask/recipeeze/datasets/rr-ratings.csv')

### Recommendation Functions

In [12]:
def mk_tbl(rows):
    #this is for creating dynamic tables
    arr = []
    for row in rows:
        title = row[2]
        r_t = row[5]
        p_t = row[3]
        c_t = row[4]
        url = row[8]
        pred = row[9]
        arr.append([pred,title,r_t,p_t,c_t,url])
    return arr

def get_r(user_id):
    # Select which system to use. Due to memory constraints, item based is the only viable option
    recommender_system = rr_model
    # N will represent how many items to recommend
    N = 2000

    # The setting to a set and back to list is a failsafe.
    rated_items = list(set(ratings_df.loc[ratings_df['user'] == user_id]['item'].tolist()))
    ratings_list = recipes_df['recipe_id'].values.tolist()
    reduced_ratings = ratings_df.loc[ratings_df['item'].isin(ratings_list)].copy()

    # Self explanitory name
    all_item_ids = list(set(reduced_ratings['item'].tolist()))

    # New_items just represents all the items not rated by the user
    new_items = [x for x in all_item_ids if x not in rated_items]

    # Estimate ratings for all unrated items
    predicted_ratings = {}
    for item_id in new_items:
        predicted_ratings[item_id] = recommender_system.predict(user_id, item_id).est
        pass

    # Get the item_ids for the top ratings
    recommended_ids = heapq.nlargest(N, predicted_ratings, key=predicted_ratings.get)
    recommended_ids = sorted(recommended_ids)

    # predicted_ratings
    recommended_df = recipes_df.loc[recipes_df['recipe_id'].isin(recommended_ids)].copy()
    #recommended_df.insert(1, 'pred_rating', np.zeros(len(recommended_ids)))
    recommended_df.insert(1, 'pred_rating', 0)

    # recommended_df = recipes_df.copy()
    for idx,item_id in enumerate(recommended_ids):
        recommended_df.iloc[idx, recommended_df.columns.get_loc('pred_rating')] = int(predicted_ratings[item_id])
        pass
    return recommended_df.head(N).sort_values('pred_rating', ascending=False)

def reg_frame(f_list,items):
    s_ = ''
    for i in items:
        str_ = f'(?=.*{i})'
        s_ += str_
    s_
    f_list = f_list[f_list['ingredients'].str.contains(fr'^\b{s_}\b',regex=True)]
    return f_list

def set_up_rr(user_id,i_list):
    f_list = get_r(user_id)
    items = i_list.split(',')
    f_list = reg_frame(f_list,items)
    f_list = f_list.reset_index(drop=True)
    f_list = f_list.T.reset_index(drop=True).T
    #f_list = f_list.head(10)
    return f_list

### Get a Recommendation Based on Ingredients

The final code that will be impletented in a cleaner fashion through the browser interface.

In [13]:
user_id = 2043209
ingredient_list = 'chocolate,cheese'
table_list = set_up_rr(user_id,ingredient_list)

In [14]:
table_list = table_list.to_numpy()
rr_recipes = pd.DataFrame(mk_tbl(table_list))
rr_recipes

,0,1,2,3,4,5
0,https://images.media-allrecipes.com/userphotos...,Chocolate Pecan Delite Recipe,45,0,15,https://www.allrecipes.com/recipe/19014
1,https://images.media-allrecipes.com/userphotos...,Double Layer Chocolate Peanut Butter Pie Recipe,260,0,0,https://www.allrecipes.com/recipe/18423
2,https://images.media-allrecipes.com/userphotos...,Easter Eggs Recipe,190,0,10,https://www.allrecipes.com/recipe/19719
3,https://images.media-allrecipes.com/userphotos...,Moon Cake Recipe,90,30,30,https://www.allrecipes.com/recipe/19895
4,https://images.media-allrecipes.com/userphotos...,Raspberry Cheesecake Recipe,255,0,0,https://www.allrecipes.com/recipe/20311
5,https://images.media-allrecipes.com/userphotos...,Sex in a Pan I Recipe,31,0,15,https://www.allrecipes.com/recipe/17073
6,https://images.media-allrecipes.com/userphotos...,Dirt Cake III Recipe,15,15,0,https://www.allrecipes.com/recipe/17540
7,https://images.media-allrecipes.com/userphotos...,'Elizabeth''s Extreme Chocolate Lover''s Cake ...,60,25,35,https://www.allrecipes.com/recipe/26027
8,https://images.media-allrecipes.com/userphotos...,Chocolate Candy Bar Cake Recipe,70,45,25,https://www.allrecipes.com/recipe/25590
9,https://images.media-allrecipes.com/userphotos...,Chocolate Caramel Cheesecake Recipe,90,0,40,https://www.allrecipes.com/recipe/25986
